# Pip installments + metrics_loading

In [ ]:
! pip install evaluate
! pip install bert_score
! pip install rouge_score
! pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 966.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 848.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=24ec7195a4512b6f2544c63a446bac02b0f236e640683260e2c49ad0b729ba80
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 37.1 MB/s eta 0:00:00
     ━━

In [ ]:
import evaluate
rouge = evaluate.load('rouge')
bleu = evaluate.load("bleu")
meteor = evaluate.load('meteor')
bertscore = evaluate.load("bertscore")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
#nltk basics
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import names
import nltk

In [ ]:
# Download the necessary resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('names')

#Import the punctuation module
from string import punctuation

import pandas as pd
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.


# Evaluation metrics

In [ ]:
def rouge_summary(df):
  summary_results = df.summary_results.tolist()
  ground_truth = df.ground_truth.tolist()

  #rouge computation (3min circa)
  values = rouge.compute(predictions = summary_results, references = ground_truth, use_aggregator= False, use_stemmer=True)

  metrics = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
  for elem in metrics:
    print(f"{elem}  mean:{np.round(np.mean(values[elem]), 4)}  std:{np.round(np.std(values[elem]), 4)} ")


def bleu_summary(df):
  summary_results = df.summary_results.tolist()
  ground_truth = df.ground_truth.tolist()

  values = [bleu.compute(predictions=[summary_results[id]], references=[[ground_truth[id]]], tokenizer=word_tokenize, smooth=True)["bleu"] for id in range(0, df.shape[0])]

  precis_0 = [bleu.compute(predictions=[summary_results[id]], references=[[ground_truth[id]]], tokenizer=word_tokenize, smooth=True)["precisions"][0] for id in range(0, df.shape[0])]
  precis_1 = [bleu.compute(predictions=[summary_results[id]], references=[[ground_truth[id]]], tokenizer=word_tokenize, smooth=True)["precisions"][1] for id in range(0, df.shape[0])]
  precis_2 = [bleu.compute(predictions=[summary_results[id]], references=[[ground_truth[id]]], tokenizer=word_tokenize, smooth=True)["precisions"][2] for id in range(0, df.shape[0])]
  precis_3 = [bleu.compute(predictions=[summary_results[id]], references=[[ground_truth[id]]], tokenizer=word_tokenize, smooth=True)["precisions"][3] for id in range(0, df.shape[0])]

  print(f"  Bleu: mean:{np.round(np.mean(values),4)}, std:{np.round(np.std(values),4)}\nunigram_mean:{np.round(np.mean(precis_0),4)}; unigram_std:{np.round(np.std(precis_0),4)}\nbigram_mean:{np.round(np.mean(precis_1),4)}; bigram_std:{np.round(np.std(precis_1),4)}\ntrigram_mean:{np.round(np.mean(precis_2),4)}; trigram_std:{np.round(np.std(precis_2),4)}\nfourgram_mean:{np.round(np.mean(precis_3),4)}; fourgram_std:{np.round(np.std(precis_3),4)}\n")


def meteor_summary(df):
  summary_results = df.summary_results.tolist()
  ground_truth = df.ground_truth.tolist()

  values = [meteor.compute(predictions=[summary_results[id]], references=[[ground_truth[id]]])["meteor"] for id in range(0, df.shape[0])]

  print(f" Meteor: mean:{np.round(np.mean(values),4)}, std:{np.round(np.std(values),4)}")


def bertscore_summary(df):
  summary_results = df.summary_results.tolist()
  ground_truth = df.ground_truth.tolist()

  values = bertscore.compute(predictions=summary_results, references=ground_truth, lang="en", model_type="distilbert-base-uncased")

  metrics = ['precision', 'recall', 'f1']
  for elem in metrics:
    print(f"{elem}  mean:{np.round(np.mean(values[elem]), 4)}  std:{np.round(np.std(values[elem]), 4)} ")


# Evaluation of the extractive summarization

Loading the dataset with the results

In [ ]:
filepath='...'

df_extractive = pd.read_csv(filepath+"/extractive_summarization_complete.csv", names=["Article","ground_truth", "summary_results"], skiprows=[0])

In [ ]:
df_extractive.head()

,Article,ground_truth,summary_results
0,"Bucharest, Romania (CNN) -- Romania's leftist ...",Leftist government files a motion with parliam...,The coalition government has come under critic...
1,"LONDON, England (CNN) -- London's newest hotel...",Entrepreneur braves recession to open rock'n'r...,"""We do not recognize there is a credit crunch ..."
2,Los Angeles (CNN) -- Actress Lindsay Lohan sho...,Lohan could have waited until June 17 to start...,"As part of Lohan's sentence, Sautner also orde..."
3,By . Associated Press . PUBLISHED: . 11:30 EST...,Gunman killed 20 first-graders and six teachin...,He wrote: 'Their most recent repugnant tactic ...
4,"(CNN) -- Finally, fans of the world's most fam...","'Harry Potter' books finally come to Kindle, N...","""I wanted to give something back to the fans t..."


In [ ]:
# Extract a random sample
sample_extractive = df_extractive.sample(n=2000, random_state=123)

Computing the metrics

In [ ]:
rouge_summary(sample_extractive)

rouge1  mean:0.2649  std:0.0938 
rouge2  mean:0.0903  std:0.072 
rougeL  mean:0.1556  std:0.0628 
rougeLsum  mean:0.2138  std:0.0825 


In [ ]:
bleu_summary(sample_extractive)

  Bleu: mean:0.0497, std:0.0425
unigram_mean:0.1747; unigram_std:0.0743
bigram_mean:0.0589; bigram_std:0.0477
trigram_mean:0.0331; trigram_std:0.0365
fourgram_mean:0.0232; fourgram_std:0.0297



In [ ]:
meteor_summary(sample_extractive)

 Meteor: mean:0.3035, std:0.1046


In [ ]:
bertscore_summary(sample_extractive)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

precision  mean:0.7405  std:0.0332 
recall  mean:0.8139  std:0.0406 
f1  mean:0.7751  std:0.0326 


# Evaluation of the abstractive summarization

Loading the abstractive summaries, previously inferenced

- 2000 articles sample

In [ ]:
filepath='...'


df_abstractive = pd.read_csv(filepath+"/Data/abstractive_sum_T5.csv")

Computing the metrics

In [ ]:
rouge_summary(df_abstractive)

rouge1  mean:0.3216  std:0.1388 
rouge2  mean:0.1295  std:0.119 
rougeL  mean:0.2281  std:0.119 
rougeLsum  mean:0.2793  std:0.1297 


In [ ]:
bleu_summary(df_abstractive)

  Bleu: mean:0.0946, std:0.0847
unigram_mean:0.3717; unigram_std:0.1481
bigram_mean:0.1462; bigram_std:0.1216
trigram_mean:0.0947; trigram_std:0.1029
fourgram_mean:0.0719; fourgram_std:0.0894



In [ ]:
meteor_summary(df_abstractive)

 Meteor: mean:0.245, std:0.1304


In [ ]:
bertscore_summary(df_abstractive)

precision  mean:0.7955  std:0.0586 
recall  mean:0.7766  std:0.0529 
f1  mean:0.7855  std:0.0526 
